# Demand analyses
Analyze the demand, both with and without demand response units in terms of
* Overall annual demand,
* Demand patterns as well as
* Demand Response utilization

## Package imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pommesevaluation.investment_results_inspection import (
    preprocess_raw_results, 
    aggregate_investment_results,
    plot_single_dispatch_pattern,
    add_area_to_existing_plot,
)
from pommesevaluation.global_vars import (
    DEMAND_RESPONSE, DEMAND_RESPONSE_RENAMED
)
from pommesevaluation.tools import update_matplotlib_params

## Define data sets and global parameters

In [ ]:
# Global settings
LANGUAGE = "German"

# Model configuration in terms of prices and costs
time_frame_in_years = 26
frequency = "1H"
dr_scenario = "50"
dr_scenarios = ["none", "5", "50", "95"]
fuel_price_scenario = "NZE"
emissions_pathway = "long-term"
impose_investment_maxima = False

# time frame to visualize
start_time_step = "2035-01-01 00:00:00"
duration_in_time_steps = 168

# inputs and outputs
path_folder_inputs = "./model_inputs/pommesinvest/"
path_folder_results = "./model_results/pommesinvest/"

file_names_inputs = {
    "demand_incl_dr_ts": f"sinks_demand_el_ts_hourly.csv",
    "demand_incl_dr_max": f"sinks_demand_el.csv",
    "demand_excl_dr_ts": f"sinks_demand_el_excl_demand_response_ts_{dr_scenario}_hourly.csv",
    "demand_excl_dr_max": f"sinks_demand_el_excl_demand_response_{dr_scenario}.csv",
    "dr_baseline": f"sinks_demand_response_el_ts_{dr_scenario}.csv",
    "dr_ava_pos": f"sinks_demand_response_el_ava_pos_ts_{dr_scenario}.csv",
    "dr_ava_neg": f"sinks_demand_response_el_ava_neg_ts_{dr_scenario}.csv",
}

filename = (
    f"investment_LP_start-2020-01-01_{time_frame_in_years}"
    f"-years_simple_freq_{frequency}"
)
if impose_investment_maxima:
    annual_investment_limits = ""
else:
    annual_investment_limits = "_no_annual_limit"
if dr_scenario != "none":
    file_add_on = (
        f"_with_dr_{dr_scenario}_"
        f"fuel_price-{fuel_price_scenario}_"
        f"co2_price-{emissions_pathway}{annual_investment_limits}"
    )
else:
    file_add_on = (
        f"_no_dr_50_"
        f"fuel_price-{fuel_price_scenario}_"
        f"co2_price-{emissions_pathway}{annual_investment_limits}"
    )
file_extension = ".csv"


NAMES = {
    "German": {
        "baseline": "Basislastgang",
        "lower_limit": "Mindestlast",
        "upper_limit": "Maximallast"
    },
    "English": {
        "baseline": "baseline demand", 
        "lower_limit": "lower demand limit",
        "upper_limit": "upper demand limit",  
    }
}
STYLE = {
    "colors": {
        NAMES[LANGUAGE]["baseline"]: "black",
        NAMES[LANGUAGE]["lower_limit"]: "#bd2020",
        NAMES[LANGUAGE]["upper_limit"]: "#bd2020"
    },
    "linestyles": {
        NAMES[LANGUAGE]["baseline"]: "-",
        NAMES[LANGUAGE]["lower_limit"]: "--",
        NAMES[LANGUAGE]["upper_limit"]: "-."
    }
}
PLOT_LABELS = {
    "German": {
        "xlabel": {
            "year": "Jahr",
            "time": "Zeit",
        },
        "ylabel": {
            "demand": "Stromnachfrage in GWh/a",
            "flex_abs": "Leistung in MW",
            "flex_rel": "Leistung normiert auf Basislastgang"
        },
        "title": "Flexibilitätsbänder"
    },
    "English": {
        "xlabel": {
            "year": "year",
            "time": "time",
        },
        "ylabel": {
            "demand": "demand in GWh/a",
            "flex_abs": "power in MW",
            "flex_rel": "power normalized to baseline"
        },
        "title": "Flexibility bands"
    },
}

path_data_out = "./data_out/"
path_plots = "./plots/"
output_file_names = {
    "demand_incl_dr_ts": "demand_incl_dr_annual",
    "demand_excl_dr_ts": "demand_exl_dr_annual",
}

# Other config
multi_header = False
rounding_precision = 2

In [ ]:
update_matplotlib_params(
    small_size=14, medium_size=16, large_size=18
)

# Annual demand inspection
## Read in and preprocess demand data

In [ ]:
data_sets = {
    data_set: pd.read_csv(f"{path_folder_inputs}{file_name}", index_col=0) 
    for data_set, file_name in file_names_inputs.items()
}

In [ ]:
# Slice 2020 data in the first place
for key in ["demand_incl_dr_ts", "demand_excl_dr_ts"]:
    data_sets[f"{key}_DE_2020"] = data_sets[key].loc[
        data_sets[key].index.str[:4] == "2020", 
        [col for col in data_sets[key].columns if "DE_" in col]
    ]

## Combine to annual values

In [ ]:
map_ts_to_max = {
    "demand_incl_dr_ts": data_sets["demand_incl_dr_max"],
    "demand_excl_dr_ts": data_sets["demand_excl_dr_max"],
}

In [ ]:
# Create annual data sets
for key in ["demand_incl_dr_ts", "demand_excl_dr_ts"]:
    data_sets[key].index = data_sets[key].index.str[:4]
    data_sets[f"{key}_annual"] = data_sets[key].groupby(data_sets[key].index).sum()["DE_sink_el_load"]
    data_sets[f"{key}_annual"] = data_sets[f"{key}_annual"].mul(
        map_ts_to_max[key].at["DE_sink_el_load", "maximum"]
    ).div(1000)
    data_sets[f"{key}_annual"].to_csv(f"{path_data_out}{output_file_names[key]}.csv")

In [ ]:
# Plot annual demand without demand response
fig, ax = plt.subplots(figsize=(12, 5))
_ = data_sets["demand_incl_dr_ts_annual"].loc["2020":"2045"].plot(kind="bar", ax=ax, color="darkblue")
_ = plt.xlabel(PLOT_LABELS[LANGUAGE]["xlabel"]["year"], labelpad=10)
_ = plt.ylabel(PLOT_LABELS[LANGUAGE]["ylabel"]["demand"], labelpad=10)
#_ = plt.legend(bbox_to_anchor=[1.02, 1.02])
current_values = plt.gca().get_yticks()
_ = plt.gca().set_yticklabels(['{:,.0f}'.format(x) for x in current_values])
plt.tight_layout()
plt.savefig(f"{path_plots}{output_file_names['demand_incl_dr_ts']}.png", dpi=300, bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
# Read in demand response potential parameter data and extract maximum value
dr_potential_data = {
    dr_cluster: pd.read_csv(
        f"{path_folder_inputs}{dr_cluster}_potential_parameters_{dr_scenario}%.csv", 
        index_col=0
    )
    for dr_cluster in DEMAND_RESPONSE
}
dr_max_potentials = {}
for key, val in dr_potential_data.items():
    dr_max_potentials[key] = val["max_cap"]

# Calculate annual demand (assumed constant here)
data_sets["dr_baseline_yearly"] = data_sets["dr_baseline"].copy()
data_sets["dr_baseline_yearly"].index = data_sets["dr_baseline_yearly"].index.str[:4]

data_sets["dr_annual_demand"] = pd.DataFrame()
for dr_cluster in DEMAND_RESPONSE:
    data_sets["dr_annual_demand"][dr_cluster] = dr_max_potentials[dr_cluster].mul(
        data_sets["dr_baseline_yearly"].groupby(
            data_sets["dr_baseline_yearly"].index
        ).sum().at["2020", dr_cluster]
    ).div(1000)

data_sets["dr_annual_demand"].index = data_sets["dr_annual_demand"].index.astype(str)
    
# Combine with demand data set excluding demand response
data_sets["demand_excl_dr_ts_annual"] = pd.concat(
    [data_sets["demand_excl_dr_ts_annual"], data_sets["dr_annual_demand"]],
    axis=1
)
data_sets["demand_excl_dr_ts_annual"].rename(
    columns={"DE_sink_el_load": "inflexible_load"},
    inplace=True
)

In [ ]:
DEMAND_RESPONSE["inflexible_load"] = "darkblue"
DEMAND_RESPONSE_RENAMED["German"]["inflexible_load"] = "inflexible Restlast"
DEMAND_RESPONSE_RENAMED["English"]["inflexible_load"] = "inflexible remaining load"

# Plot annual demand excl demand response with demand response baseline demand on top
fig, ax = plt.subplots(figsize=(12, 7))
data_sets["demand_excl_dr_ts_annual"] = data_sets["demand_excl_dr_ts_annual"][[col for col in DEMAND_RESPONSE]]
to_plot = data_sets["demand_excl_dr_ts_annual"].rename(columns=DEMAND_RESPONSE_RENAMED[LANGUAGE])
colors = {
    DEMAND_RESPONSE_RENAMED[LANGUAGE][c]: DEMAND_RESPONSE[c] 
    for c in DEMAND_RESPONSE_RENAMED[LANGUAGE] 
    if DEMAND_RESPONSE_RENAMED[LANGUAGE][c] in to_plot.columns
}

# Sort columns
to_plot = to_plot[[
    DEMAND_RESPONSE_RENAMED[LANGUAGE]["inflexible_load"]
] + [
    DEMAND_RESPONSE_RENAMED[LANGUAGE][col] 
    for col in DEMAND_RESPONSE_RENAMED[LANGUAGE] 
    if col != "inflexible_load"]
]
_ = to_plot.loc["2020":"2045"].plot(kind="bar", edgecolor="#2A2A2A", stacked=True, ax=ax, color=colors)
_ = plt.xlabel(PLOT_LABELS[LANGUAGE]["xlabel"]["year"], labelpad=10)
_ = plt.ylabel(PLOT_LABELS[LANGUAGE]["ylabel"]["demand"], labelpad=10)
_ = plt.legend(loc='upper center', bbox_to_anchor=[0.5, -0.25], fancybox=True, shadow=False, ncol=2)
current_values = plt.gca().get_yticks()
_ = plt.gca().set_yticklabels(['{:,.0f}'.format(x) for x in current_values])
plt.tight_layout()
plt.savefig(f"{path_plots}{output_file_names['demand_excl_dr_ts']}.png", dpi=300)
plt.show()
plt.close()

_ = DEMAND_RESPONSE.pop("inflexible_load")
_ = DEMAND_RESPONSE_RENAMED["German"].pop("inflexible_load")
_ = DEMAND_RESPONSE_RENAMED["English"].pop("inflexible_load")

# Demand patterns inspection
## Extract demand response investments results

In [ ]:
investment_results_raw = pd.read_csv(f"{path_folder_results}{filename}{file_add_on}_investment{file_extension}", index_col=0)
processed_results = preprocess_raw_results(investment_results_raw)
aggregated_results, other_storage_results = aggregate_investment_results(
    processed_results, energy_carriers=[], by="energy_carrier"
)

demand_response_investments = aggregated_results.loc[
    aggregated_results.index.get_level_values(0).isin(DEMAND_RESPONSE), "total"
]

## Calculate profiles

Extract the absolute values of
* baseline profile,
* economic downshift potential and
* economic upshift potential.

Calculate the absolute demand bounds from downshift resp. upshift potentials.

> _Note:_
> * _Downshift is limited to baseline profile._
> * _This is implicitly done in pommesinvest by setting lower bound of flows to 0_

In [ ]:
profiles = {}
for dr_cluster in DEMAND_RESPONSE:
    profiles[dr_cluster] = pd.DataFrame(
        columns=["baseline", "downshift_potential", "upshift_potential"]
    )
    to_concat = {k: [] for k in profiles[dr_cluster].columns}
    
    # Calculate absolute values using annual max capacity resp. potentials
    for iter_year in range(2020, 2046):
        to_concat["baseline"].append(
            data_sets["dr_baseline"].loc[
                f"{iter_year}-01-01 00:00": f"{iter_year}-12-31 23:59", dr_cluster
            ].mul(dr_max_potentials[dr_cluster].loc[iter_year])
        )
        to_concat["downshift_potential"].append(
           data_sets["dr_ava_pos"].loc[
                f"{iter_year}-01-01 00:00": f"{iter_year}-12-31 23:59", dr_cluster
            ].mul(demand_response_investments.loc[dr_cluster, str(iter_year)])
        )
        to_concat["upshift_potential"].append(
           data_sets["dr_ava_neg"].loc[
                f"{iter_year}-01-01 00:00": f"{iter_year}-12-31 23:59", dr_cluster
            ].mul(demand_response_investments.loc[dr_cluster, str(iter_year)])
        ) 

    for col in profiles[dr_cluster].columns:
        profiles[dr_cluster][col] = pd.concat(to_concat[col])
    
    # Limit downshift potential
    profiles[dr_cluster]["downshift_potential"] = np.where(
        profiles[dr_cluster]["baseline"] < profiles[dr_cluster]["downshift_potential"],
        profiles[dr_cluster]["baseline"],
        profiles[dr_cluster]["downshift_potential"]
    )
    profiles[dr_cluster]["lower_limit"] = profiles[dr_cluster]["baseline"] - profiles[dr_cluster]["downshift_potential"]
    profiles[dr_cluster]["upper_limit"] = profiles[dr_cluster]["baseline"] + profiles[dr_cluster]["upshift_potential"]
    
    profiles[dr_cluster] = profiles[dr_cluster][[col for col in NAMES[LANGUAGE]]].rename(columns=NAMES[LANGUAGE])

## Plot absolute profiles (baseline and flexibility band)

In [ ]:
for dr_cluster in DEMAND_RESPONSE:
    fig, ax = plot_single_dispatch_pattern(
        profiles[dr_cluster],
        start_time_step=start_time_step,
        amount_of_time_steps=duration_in_time_steps,
        colors=STYLE["colors"],
        save=False,
        path_plots="./plots/",
        filename=f"absolute_potential_{dr_cluster}",
        kind="line",
        stacked=None,
        figsize=(15, 10),
        linestyle=STYLE["linestyles"],
        title=PLOT_LABELS[LANGUAGE]["title"],
        ylabel=PLOT_LABELS[LANGUAGE]["ylabel"]["flex_abs"],
        return_plot=True,
        bbox_params=[0.5, -0.45],
        language="German",
    )
    # Add a flex band
    profiles[dr_cluster]["flex_band"] = profiles[dr_cluster][NAMES[LANGUAGE]["upper_limit"]] - profiles[dr_cluster][NAMES[LANGUAGE]["lower_limit"]]
    add_area_to_existing_plot(
        data=profiles[dr_cluster][[NAMES[LANGUAGE]["lower_limit"], "flex_band"]], 
        start_time_step=start_time_step, 
        amount_of_time_steps=duration_in_time_steps, 
        colors={NAMES[LANGUAGE]["lower_limit"]: "#ffffff", "flex_band": "lightblue"},
        ax=ax,
        save=True,
        path_plots="./plots/",
        filename=f"absolute_potential_{dr_cluster}",
        bbox_params=[0.5, -0.45],
    )
    profiles[dr_cluster].drop(columns=["flex_band"], inplace=True)

## Calculate and plot relative profiles
For normalization, define maximum of baseline demand of respective time frame to equal 1

In [ ]:
# get end index to derive time frame for normalization
index_start = int(profiles[dr_cluster].index.get_loc(start_time_step))
index_end = int(index_start + duration_in_time_steps)
end_time_step = profiles[dr_cluster].iloc[index_end].name

relative_profiles = {}
for dr_cluster in DEMAND_RESPONSE:
    relative_profiles[dr_cluster] = (
        profiles[dr_cluster].iloc[index_start : index_end].div(
            profiles[dr_cluster].iloc[index_start : index_end][NAMES[LANGUAGE]["baseline"]].max()
        )
    )
    
    fig, ax = plot_single_dispatch_pattern(
        relative_profiles[dr_cluster],
        start_time_step=start_time_step,
        amount_of_time_steps=duration_in_time_steps - 2,
        colors=STYLE["colors"],
        save=False,
        path_plots="./plots/",
        filename=f"relative_potential_{dr_cluster}",
        kind="line",
        stacked=None,
        figsize=(15, 10),
        linestyle=STYLE["linestyles"],
        title=PLOT_LABELS[LANGUAGE]["title"],
        ylabel=PLOT_LABELS[LANGUAGE]["ylabel"]["flex_rel"],
        return_plot=True,
        bbox_params=[0.5, -0.5],
        language="German",
    )
    # Add a flex band
    relative_profiles[dr_cluster]["flex_band"] = relative_profiles[dr_cluster][NAMES[LANGUAGE]["upper_limit"]] - relative_profiles[dr_cluster][NAMES[LANGUAGE]["lower_limit"]]
    add_area_to_existing_plot(
        data=relative_profiles[dr_cluster][[NAMES[LANGUAGE]["lower_limit"], "flex_band"]], 
        start_time_step=start_time_step, 
        amount_of_time_steps=duration_in_time_steps - 2, 
        colors={NAMES[LANGUAGE]["lower_limit"]: "#ffffff", "flex_band": "lightblue"},
        ax=ax,
        save=True,
        path_plots="./plots/",
        filename=f"relative_potential_{dr_cluster}",
        bbox_params=[0.5, -0.5],
    )
    relative_profiles[dr_cluster].drop(columns=["flex_band"], inplace=True)

# Analyze demand response utilization
## Read in production results from model and filter for demand response columns

In [ ]:
if multi_header:
    header = [0, 1]
else:
    header = 0
production_results_raw = pd.read_csv(
    f"{path_folder_results}{filename}{file_add_on}_production{file_extension}", index_col=0, header=header
).T
processed_results = preprocess_raw_results(
    production_results_raw, investments=False, multi_header=multi_header
).drop(columns="year").round(rounding_precision)
aggregated_results = aggregate_investment_results(
    processed_results, energy_carriers=[], by="energy_carrier", investments=False
).T
del production_results_raw, processed_results

all_demand_response_cols = list(set([
    col for col in aggregated_results.columns for key in DEMAND_RESPONSE if key in col
]))

demand_response_after_cols = [col for col in all_demand_response_cols if "_demand_after" in col]
demand_response_other_cols = [
    col for col in all_demand_response_cols 
    if col not in demand_response_after_cols
    # Exclude fictious demand response storage level which can be calculated ex post
    and not "storage_level" in col
]
demand_response_pattern = aggregated_results[demand_response_other_cols]

In [ ]:
downshift_cols = [col for col in demand_response_pattern.columns if "dsm_do_shift" in col]
upshift_cols = [col for col in demand_response_pattern.columns if "dsm_up" in col]
shed_cols = [col for col in demand_response_pattern.columns if "dsm_do_shed" in col]

overall_downshift = pd.DataFrame()
overall_upshift = pd.DataFrame()
overall_shedding = pd.DataFrame()
for iter_year in demand_response_pattern.index.str[:4].unique():
    overall_downshift.loc[iter_year, downshift_cols] = demand_response_pattern.loc[[idx for idx in demand_response_pattern.index if idx[:4] == iter_year], downshift_cols].sum()
    overall_upshift.loc[iter_year, upshift_cols] = demand_response_pattern.loc[[idx for idx in demand_response_pattern.index if idx[:4] == iter_year], upshift_cols].sum()
    overall_shedding.loc[iter_year, shed_cols] = demand_response_pattern.loc[[idx for idx in demand_response_pattern.index if idx[:4] == iter_year], shed_cols].sum()

In [ ]:
overall_downshift.columns = [col[0] for col in overall_downshift.columns.str.rsplit("_", 3)]
overall_shedding.columns = [col[0] for col in overall_shedding.columns.str.rsplit("_", 3)]
overall_upshift.columns = [col[0] for col in overall_upshift.columns.str.rsplit("_", 2)]

overall_downshift.to_csv(f"{path_data_out}overall_shifting_in_mwh_dr_scen_{dr_scenario}.csv", sep=";")
overall_shedding.to_csv(f"{path_data_out}overall_shedding_in_mwh_dr_scen_{dr_scenario}.csv", sep=";")

# Check for balancing
diff = pd.DataFrame()
for col in overall_downshift.columns:
    diff[col] = overall_downshift[col] - overall_upshift[col]
diff / overall_downshift
assert (diff.min() < 1e-4).all()